In [17]:
import pandas as pd
import os, codecs

# create df
df_liar = pd.read_csv("train.tsv", encoding="utf8", sep="\t", names=["id", "truth-value", 
                                                                     "text", "topic", "name", "job", 
                                                                     "state", "politics", "count1", "count2", 
                                                                     "count3", "count4", "count5", "context"])

df_liar.head(3)

,id,truth-value,text,topic,name,job,state,politics,count1,count2,count3,count4,count5,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver


In [4]:
def classify(text):
    if text == "false":
        return 0
    elif text == "true":
        return 1 
    else: 
        return -1 

In [12]:
# add the class of truth-values
df_liar["class"] = df_liar["truth-value"].apply(classify) 

In [16]:
# reduce the dataset to only true/false sentences
df_reduced = df_liar[df_liar["class"] != -1]
print(df_reduced.shape)
df_reduced.head(3)

(3671, 15)


,id,truth-value,text,topic,name,job,state,politics,count1,count2,count3,count4,count5,context,class
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,0
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,0
5,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,1


In [25]:
# TF-IDF representation of the text
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train = count_vect.fit(df_reduced.text)
X_train = count_vect.transform(df_reduced.text)
# --> for the test you use same vectorizer, but just transform (not fit!)
X_train.shape

(3671, 7451)

In [26]:
# subtract the classes
y_train = df_reduced["class"].values

In [27]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs')

# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X_train, y_train)

/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
df_liar_test = pd.read_csv("test.tsv", encoding="utf8", sep="\t", names=["id", "truth-value", 
                                                                     "text", "topic", "name", "job", 
                                                                     "state", "politics", "count1", "count2", 
                                                                     "count3", "count4", "count5", "context"])
df_liar_test["class"] = df_liar_test["truth-value"].apply(classify)
df_test_reduced = df_liar_test[df_liar_test["class"] != -1]

X_test = count_vect.transform(df_test_reduced.text) 
y_test = df_test_reduced["class"].values
X_test.shape

(457, 7451)

In [29]:
# we could try to evaluate the model
logreg.fit(X_train, y_train)
y_hat_test = logreg.predict(X_test)

# evaluate using accuracy: proportion of correctly predicted over total
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_hat_test))
print(accuracy_score(y_test, y_hat_test, normalize=False))

0.599562363239
274


/Applications/Anaconda/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [41]:
print(logreg.coef_)
print(logreg.coef_.shape)

[[-0.17203638 -0.27699619  0.05011177 ..., -0.15039476  0.27433863
   0.47206197]]
(1, 7451)


In [50]:
print(count_vect.vocabulary_.keys())

dict_keys(['says', 'the', 'annies', 'list', 'political', 'group', 'supports', 'third', 'trimester', 'abortions', 'on', 'demand', 'health', 'care', 'reform', 'legislation', 'is', 'likely', 'to', 'mandate', 'free', 'sex', 'change', 'surgeries', 'chicago', 'bears', 'have', 'had', 'more', 'starting', 'quarterbacks', 'in', 'last', '10', 'years', 'than', 'total', 'number', 'of', 'tenured', 'uw', 'faculty', 'fired', 'during', 'two', 'decades', 'when', 'mitt', 'romney', 'was', 'governor', 'massachusetts', 'we', 'didnt', 'just', 'slow', 'rate', 'growth', 'our', 'government', 'actually', 'cut', 'it', 'mccain', 'opposed', 'requirement', 'that', 'buy', 'american', 'made', 'motorcycles', 'and', 'he', 'said', 'all', 'provisions', 'were', 'quote', 'disgraceful', 'women', 'men', 'both', 'are', 'making', 'less', 'you', 'adjust', 'for', 'inflation', 'john', 'kitzhaber', 'first', 'elected', 'dont', 'know', 'who', 'jonathan', 'gruber', 'isis', 'supporter', 'tweeted', 'at', '34', 'shooting', 'began', '45',